In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install torch torchvision torchaudio cpuonly

  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.6.0-cp311-cp311-win_amd64.whl.metadata (6.7 kB)


ERROR: Could not find a version that satisfies the requirement cpuonly (from versions: none)
ERROR: No matching distribution found for cpuonly

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoModel, AutoTokenizer

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
from transformers import ModernBertModel

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ModernBertModel
import dill

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class ModernBertClassifier(nn.Module):
    def __init__(self, num_labels):
        super(ModernBertClassifier, self).__init__()
        self.bert = ModernBertModel.from_pretrained("answerdotai/ModernBERT-base")

        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.LayerNorm(512),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Dropout(0.3),
            
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.classifier(outputs.last_hidden_state[:, 0, :])

    def predict(self, text, label_encoder=None):  
        encoding = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(next(self.parameters()).device)
        with torch.no_grad():
            outputs = self(**encoding)
            logits = outputs.cpu()
            predictions = torch.argmax(logits, dim=-1)

        try:
            predicted_label = label_encoder.classes_[predictions[0]] if label_encoder is not None else predictions[0].item()
        except NameError:
            predicted_label = predictions[0].item()

        return predicted_label,outputs

model_path = r'C:\Users\Dell\Downloads\Data Science Stuff\Machine Learning\Internship Tasks\Sentimental Analysis Model\model.pth'
model = torch.load(model_path, map_location=device, pickle_module=dill)
model.tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
model.eval()

OptimizedModule(
  (_orig_mod): ModernBertClassifier(
    (bert): ModernBertModel(
      (embeddings): ModernBertEmbeddings(
        (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
        (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (layers): ModuleList(
        (0): ModernBertEncoderLayer(
          (attn_norm): Identity()
          (attn): ModernBertAttention(
            (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
            (rotary_emb): ModernBertRotaryEmbedding()
            (Wo): Linear(in_features=768, out_features=768, bias=False)
            (out_drop): Identity()
          )
          (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): ModernBertMLP(
            (Wi): Linear(in_features=768, out_features=2304, bias=False)
            (act): GELUActivation()
            (drop): Dropout(p=0.0, inplace=False)
            (Wo): Linear(in_fea

In [7]:
emotion_labels = ['Aggressive','Happy','Sad','Neutral','Fear']
import re
def basic_text_cleaning(text):
    text = re.sub(r'\s+', ' ', text.lower()).split()
    return text

In [8]:
text = "My heart started racing the moment I heard the footsteps behind me. It was late at night, and the street was almost empty. I told myself I was just being paranoid, but the uneasy feeling in my stomach wouldn’t go away. I picked up my pace, but the footsteps matched mine. My hands trembled as I reached for my phone, praying nothing bad was about to happen."
cleaned_text = basic_text_cleaning(text)
predicted_sentiment_index, labels = model.predict(cleaned_text)
print(f"Predicted Sentiment Index: {predicted_sentiment_index}")
print(f"Predicted Label: {labels}")

Compiling the model with `torch.compile` and using a `torch.cpu` device is not supported. Falling back to non-compiled mode.


Predicted Sentiment Index: Neutral
Predicted Label: {'Aggressive': 6.95, 'Fear': 2.6, 'Happy': 17.42, 'Neutral': 67.7, 'Sad': 5.33}


In [9]:
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
texts = [
    ("Oh fuck, it’s coming!", "Fear"),
    ("Hell yeah, I won!", "Happiness"),
    ("I miss you so much.", "Sadness"),
    ("You fucking suck!", "Aggression"),
    ("It’s just another day.", "Neutrality"),
    ("Something’s in the shadows!", "Fear"),
    ("Best day ever, fuck yes!", "Happiness"),
    ("He’s never coming back.", "Sadness"),
    ("I’ll kick your ass!", "Aggression"),
    ("Weather’s fine, I guess.", "Neutrality"),
    ("Footsteps behind me—shit!", "Fear"),
    ("Sunshine and smiles today!", "Happiness"),
    ("My heart’s fucking shattered.", "Sadness"),
    ("Piss off, you prick!", "Aggression"),
    ("Nothing happened, all good.", "Neutrality"),
    ("The dark hallway stretched forever, and I heard breathing—fuck, I’m not alone!", "Fear"),
    ("Got a raise today, celebrated with friends, life’s fucking awesome right now!", "Happiness"),
    ("Her photo fell off the shelf, and I cried for hours, lost.", "Sadness"),
    ("You screwed me over again, you bastard—I’ll make you regret it!", "Aggression"),
    ("Went to work, ate lunch, came home—same old routine, nothing special.", "Neutrality"),
    ("A shadow darted across the room, my pulse raced, what the hell?", "Fear"),
    ("The beach was perfect—sun, waves, beer—couldn’t ask for more joy!", "Happiness"),
    ("Found his old guitar, strummed it, and felt the emptiness hit hard.", "Sadness"),
    ("Fuck you, you’re dead to me—I’ll smash everything you love!", "Aggression"),
    ("Drove through town, saw some birds, it was an average day.", "Neutrality"),
    ("Something scratched the door at midnight, I froze, too scared to check.", "Fear"),
    ("Kids surprised me with breakfast—pancakes and hugs, best morning in ages!", "Happiness"),
    ("The hospital called, she’s gone, and I’m drowning in this silence.", "Sadness"),
    ("You’re a fucking moron, I’ll rip your throat out—try me!", "Aggression"),
    ("Sat by the window, read a bit, watched the rain—pretty calm.", "Neutrality"),
    ("I was alone in the house when the lights flickered and died, leaving me in pitch black. Then came the tapping—soft at first, then louder, from the attic. My heart hammered as I grabbed a bat, every creak sounding like doom. Fuck, something’s up there, waiting.", "Fear"),
    ("The party was insane—lights everywhere, music shaking the walls, and I danced with strangers who felt like friends. Won a raffle, ate cake, laughed until my sides hurt—pure fucking happiness, like the world was mine for a night!", "Happiness"),
    ("I found her diary under the bed, pages filled with dreams we’d never share. She’s gone now, taken by that damn crash, and I’m left reading her words, tears blurring the ink, wishing I’d said I loved her one more time.", "Sadness"),
    ("You fucked up my project, you incompetent asshole—I spent weeks on it! I’ll smash your face into the desk, you worthless prick, and watch you bleed while I laugh. Don’t ever cross me again!", "Aggression"),
    ("Woke up late, made coffee, walked the dog around the block. Saw some neighbors, waved, nothing big. Came home, watched a movie, ate leftovers—typical day, nothing to write home about, just steady.", "Neutrality"),
    ("The forest was too quiet, then a branch snapped behind me—loud, close. I turned, saw nothing, but felt eyes boring into me. My hands shook, sweat stung my eyes, fuck, I’m not safe here!", "Fear"),
    ("Got tickets to the game, front row—team won in overtime, crowd went wild! Beers flowed, high-fives all around, I’m still buzzing—fucking perfect day, nothing beats this high!", "Happiness"),
    ("His room’s untouched since he left—toys, posters, all frozen in time. I sit there sometimes, staring, feeling the weight of his absence crush me. He was my son, and I failed him.", "Sadness"),
    ("You’re a fucking disgrace, ruining everything with your stupidity—I’ll break your jaw, you sniveling shit! Step up again, and I’ll bury you, no one fucks with me like this!", "Aggression"),
    ("Drove to the lake, fished for a bit, caught nothing but enjoyed the quiet. Sun was out, breeze was cool—decent day, nothing wild, just peaceful enough to relax.", "Neutrality"),
    ("I’d been hiking for hours when the sun dipped below the ridge, casting the valley in shadow. The trail was supposed to be safe, but the silence felt wrong—too thick, too heavy. Then I heard it: a low, guttural hum, not animal, not human, vibrating through the trees. My flashlight flickered, and I stumbled, heart pounding like it’d burst. Shapes moved in the dark—tall, thin, too many limbs—circling me. I ran, branches clawing my face, breath ragged, but the hum followed, closer, whispering my name in a voice that wasn’t there. I made it to the car, locked the doors, and peeled out, but in the rearview, I saw them—fuck, they’re still out there, watching.", "Fear"),
    ("The day started with a call—I’d won a trip to Hawaii, all expenses paid! I screamed, danced around the kitchen, called my mom, my best friend, everyone. Packed my bags in a blur, grinning like an idiot, and the flight was a dream—champagne, clouds, the works. Landed to ukuleles and leis, the beach stretching out like paradise. Swam with turtles, ate pineapple so sweet it hurt, laughed with locals until midnight. Fireworks lit the sky, and I stood there, sand between my toes, feeling like the luckiest bastard alive—pure, unfiltered joy, every second a fucking gift!", "Happiness"),
    ("She was my anchor, the one who kept me steady through every storm, and now she’s gone—cancer took her in six brutal months. I watched her fade, her smile dimming, her hands growing cold in mine. I promised I’d be strong, but it was a lie. Now I’m in our house, her clothes still in the closet, her voice echoing in my head. I found her favorite scarf today, held it, and broke—sobs racking me until I couldn’t breathe. The kids don’t know how to help, and I don’t either. Every room’s a memory, every silence a wound, and I’m lost in this grief, drowning in what we’ll never have again.", "Sadness"),
    ("That fucker stole my deal—months of work, gone because of his slimy hands! I stormed his office, kicked the door in, and grabbed him by the collar. ‘You’re dead, you piece of shit!’ I yelled, slamming him against the wall, fists flying. He begged, blood dripping, but I didn’t stop—pounded his face until my knuckles bled, rage burning through me. ‘Fuck with me again, and I’ll end you!’ I spat, leaving him crumpled. No one screws me over and walks away—I’m the goddamn storm now!", "Aggression"),
    ("Got up at seven, made toast, checked the news—same old headlines, nothing new. Walked to the store, bought milk, chatted with the cashier about the weather. Came home, mowed the lawn, watched cars pass by. Lunch was a sandwich, dinner was pasta, nothing fancy. Read a chapter of a book, scrolled my phone, went to bed early. It was a day like any other—quiet, uneventful, just rolling along without much fuss or fanfare.", "Neutrality"),
    ("The closet door creaked open by itself. I hadn’t touched it.", "Fear"),
    ("Promotion, bonus, and a new car—life’s a fucking dream!", "Happiness"),
    ("Empty chair at the dinner table. No one mentions her anymore.", "Sadness"),
    ("I’ll burn your house down, you traitorous snake!", "Aggression"),
    ("Slept in, watered the plants, took a nap. Zero drama.", "Neutrality"),
    ("The phone rang at 3 AM. No one was on the line.", "Fear"),
    ("Surprise party! Balloons, cake, everyone I love—I’m so fucking happy!", "Happiness"),
    ("His laughter still echoes in this empty room.", "Sadness"),
    ("Say another word and I’ll shut your mouth permanently.", "Aggression"),
    ("Brewed tea, listened to the radio, nothing remarkable.", "Neutrality"),
    ("The figure in the window wasn’t there when I turned around.", "Fear"),
    ("First place in the tournament! Trophy in hand, crowd roaring—euphoria!", "Happiness"),
    ("Her perfume lingers on the pillow. I can’t wash it.", "Sadness"),
    ("I’ll gut you like a fish, you backstabbing coward!", "Aggression"),
    ("Cloudy skies, light traffic, ordinary Tuesday.", "Neutrality"),
    ("The whisper came from the empty passenger seat.", "Fear"),
    ("Engagement ring, tears, ‘Yes!’—best moment of my life.", "Happiness"),
    ("The last text he sent: ‘See you soon.’ He never came.", "Sadness"),
    ("I’ll carve my name into your bones, you pathetic worm!", "Aggression"),
    ("Tidied the garage, sorted old clothes. No surprises.", "Neutrality"),
    ("Fuck off!", "Aggression"),
    ("So happy!", "Happiness"),
    ("Heart’s breaking.", "Sadness"),
    ("You’re trash.", "Aggression"),
    ("Just okay.", "Neutrality"),
    ("Shadows moved fast, and I froze.", "Fear"),
    ("Sun’s out, I’m fucking pumped!", "Happiness"),
    ("She’s gone, and I’m empty inside.", "Sadness"),
    ("I’ll smash your fucking face in!", "Aggression"),
    ("Day’s alright, nothing special.", "Neutrality"),
    ("Something’s watching me from the dark, fuck!", "Fear"),
    ("Won a prize—life’s goddamn great!", "Happiness"),
    ("Lost him, and it’s killing me slow.", "Sadness"),
    ("You’re a useless piece of shit!", "Aggression"),
    ("Rain fell, I just sat there.", "Neutrality"),
    ("The creaking upstairs won’t stop—oh shit, I’m scared as hell!", "Fear"),
    ("Party rocked, danced all night, pure bliss!", "Happiness"),
    ("He died, and I can’t stop crying, fuck this pain.", "Sadness"),
    ("Get out, or I’ll break your neck, asshole!", "Aggression"),
    ("Watched TV, ate snacks, pretty chill day.", "Neutrality"),
    ("I heard whispers in the walls last night, faint but fucking real. My hands shook as I grabbed a knife, heart racing like a drum. Was it the wind? Bullshit—it said my name.", "Fear"),
    ("Woke up to birds chirping, coffee brewing, and a big-ass grin on my face—best morning ever, no contest!", "Happiness"),
    ("She left me with nothing but a note, and now I’m staring at it, tears soaking the paper, feeling like the world’s caving in.", "Sadness"),
    ("You fucked up again, you dumb bastard—I’ll rip your head off and shove it up your ass!", "Aggression"),
    ("Walked the park, saw some ducks, it was fine, nothing wild.", "Neutrality"),
    ("The fog rolled in thick, and I swear I saw eyes glowing in it—red, unblinking, tracking me. I ran, tripping over roots, breath hitching, knowing I wasn’t alone out there.", "Fear"),
    ("Got a promotion today, and the whole office cheered—fuck yeah, I’m on top of the world, unstoppable!", "Happiness"),
    ("I stood by his grave, rain pounding down, and all I could think was how I should’ve been there—should’ve saved him. The guilt’s a chain around my neck, dragging me under every damn day.", "Sadness"),
    ("You think you’re hot shit? I’ll beat you bloody, you whining prick—try me!", "Aggression"),
    ("Sat on the porch, sipped beer, watched the sunset—decent enough.", "Neutrality"),
    ("Last night, the power cut out, and something scratched at my window—slow, deliberate, like it wanted me to hear it. I didn’t sleep, just sat there, shotgun in hand, waiting for the glass to shatter and whatever the fuck it was to come in. My dog whimpered all night, and I couldn’t shake the feeling it wasn’t human, wasn’t animal—just wrong.", "Fear"),
    ("The festival hit like a goddamn dream—lights flashing, music thumping, everyone laughing and hugging. I ate cotton candy, won a stupid stuffed bear, and danced until my feet screamed—pure fucking joy from start to finish!", "Happiness"),
    ("I found her old sweater in the closet today, and it hit me like a truck—she’s never coming back. I held it, smelling her faded perfume, and broke down, sobbing into the fabric. We had plans, dreams, a whole life ahead, but now it’s just me, alone, staring at ghosts in every corner of this empty house.", "Sadness"),
    ("You’re a fucking disgrace, screwing up everything you touch—I’ll pound you into the dirt, you sniveling little bitch, and laugh while you bleed!", "Aggression"),
    ("Drove to the store, bought some bread, came home—nothing exciting, just another day ticking by.", "Neutrality"),
    ("I woke up to a noise in the basement, a low thud that rattled the floorboards, and my stomach dropped like a stone. I grabbed a flashlight, legs shaking, and crept down the stairs, each step creaking louder than the last. The air got cold, too cold, and then I saw it—a shape in the corner, tall and twisted, with too many arms, just standing there, silent. I bolted, slamming the door, but I can still hear it down there, tapping, waiting, and I don’t know if I’ll make it through the night—fuck, I’m terrified.", "Fear"),
    ("Today was the best damn day in years—sun shining, kids playing outside, and I landed a fat bonus at work. I grilled burgers, cracked open a beer, and watched the neighborhood buzz with life, feeling like nothing could touch me, like I’d finally won at something big!", "Happiness"),
    ("He was my brother, my rock, and now he’s just ashes in a box because I didn’t pick up the phone that night. I replay it over and over—his voice begging me to answer, me too drunk to care. The crash took him, and I’m left here, a hollow shell, drowning in whiskey and regret, hating myself more every second I breathe without him.", "Sadness"),
    ("You’ve got some nerve, you cocky fuck—I’ll tear you apart, limb by limb, and spit on your corpse, you miserable waste of space!", "Aggression"),
    ("Went fishing, caught nothing, came back—meh, it was whatever, no big deal.", "Neutrality"),
    ("The woods were dead quiet last night, not a bird, not a breeze, just this heavy, suffocating stillness that made my skin prickle. I was camping alone, stupidly thinking it’d be peaceful, when I heard it—a snap, then a growl, too close, too deep. I shone my light, and fuck me, there were eyes, dozens of them, glinting in the trees, circling. I grabbed my axe, chest tight, knowing I couldn’t outrun whatever was out there. It didn’t attack, just watched, but I didn’t sleep—couldn’t, not with that weight pressing down, like they were waiting for me to break. I packed up at dawn, swearing I’d never go back, but I still feel them, even now, like they followed me home.", "Fear"),
    ("I won the lottery—fucking jackpot, half a million bucks! I screamed, jumped around like a lunatic, called everyone I know, and popped champagne until I couldn’t see straight. The whole street partied with me, music blasting, neighbors I’d never met hugging me like family—it was chaos, beautiful chaos, and I’m still buzzing, high on life, ready to live like a king!", "Happiness"),
    ("She was my everything, my wife, my light, and I lost her to that damn disease in six months flat. I sat by her bed, holding her hand as she faded, promising I’d be okay, lying through my teeth because I knew I wouldn’t. Now I’m here, in this house we built, surrounded by her things—her books, her scarf, that stupid mug she loved—and it’s killing me. Every memory’s a blade, every silence a scream, and I can’t escape it. I tried to move on, but I see her in every shadow, hear her in every creak, and I’m falling apart, piece by piece, until there’s nothing left but this endless, aching void where she used to be.", "Sadness"),
    ("You’re a fucking idiot, screwing up my day again—I’ll bash your skull in, you whining, gutless prick, and dance on your broken bones!", "Aggression"),
    ("Read a book, took a nap, watched the clouds—quiet day, nothing to rave about, nothing to hate.", "Neutrality"),
     ("Woo! Got the job!", "Happy"),
    ("Sunshine, ice cream, and laughter—perfect day!", "Happy"),
    ("She said yes! Engagement ring shining on her finger, and I’ve never been prouder.", "Happy"),
    ("Puppies everywhere! Tiny paws, wet noses, pure joy.", "Happy"),
    ("Promotion + bonus = best Friday ever!", "Happy"),
    ("Surprise reunion with my childhood best friend—hugs, tears, and endless stories.", "Happy"),
    ("First sip of coffee this morning hit different. Today’s gonna be great.", "Happy"),
    ("Danced in the rain like nobody watched. Soaked and happy.", "Happy"),
    ("Kids made me a ‘Best Dad’ card. Heart melted.", "Happy"),
    ("Beach day—sand, waves, and zero responsibilities. Bliss.", "Happy"),
    ("Finished my novel! Years of work, finally done. Champagne time.", "Happy"),
    ("Random stranger paid for my coffee. Faith in humanity restored.", "Happy"),
    ("Won front-row concert tickets! Screamed like a teenager.", "Happy"),
    ("Family BBQ, dad’s jokes, mom’s pie—nothing beats this.", "Happy"),
    ("Adopted a rescue dog today. Meet my new shadow!", "Happy"),
    ("Saw a double rainbow after the storm. Magic.", "Happy"),
    ("Best friend’s baby giggled at me for the first time. Instant love.", "Happy"),
    ("Ate the spiciest ramen and survived. Victory dance ensued.", "Happy"),
    ("Grandma’s 90th birthday—three generations laughing in one room.", "Happy"),
    ("Found $20 in old jeans. Small wins count!", "Happy")
]

In [ ]:
texts = [
        # Sad (20 texts)
    ("Empty bed again.", "Sad"),
    ("The last ‘I love you’ was six months ago.", "Sad"),
    ("His jacket still smells like him. Can’t bring myself to wash it.", "Sad"),
    ("Cancelled plans. Again. Nobody notices.", "Sad"),
    ("Scrolled through our old texts. Deleted the chat. Cried.", "Sad"),
    ("Anniversary today. He’s not here to celebrate.", "Sad"),
    ("Hospital walls are too white. Too quiet. Too final.", "Sad"),
    ("Dog’s bowl is still full. She’s not coming back.", "Sad"),
    ("‘We need to talk.’ Those words ruined everything.", "Sad"),
    ("Playlist shuffled to our song. Had to pull over.", "Sad"),
    ("They sold our childhood home. Memories packed away.", "Sad"),
    ("Forgot to call Mom. Now it’s too late.", "Sad"),
    ("Watched her walk away with someone else.", "Sad"),
    ("The funeral was sunny. Ironic.", "Sad"),
    ("Texts left on ‘read’ for weeks.", "Sad"),
    ("Found her hairpin under the couch. Didn’t realize I was still looking.", "Sad"),
    ("His side of the closet is empty now.", "Sad"),
    ("Missed his graduation. Flight got delayed forever.", "Sad"),
    ("Plant died. Even the cactus couldn’t survive me.", "Sad"),
    ("‘You’ll get over it.’ No, I won’t.", "Sad"),

    # Fear (20 texts)
    ("Something’s under the bed.", "Fear"),
    ("The lights flickered. I’m alone in this house.", "Fear"),
    ("Footsteps followed me home. No one was there.", "Fear"),
    ("Woke up to whispering. My phone was off.", "Fear"),
    ("Shadow moved across the wall. No source.", "Fear"),
    ("Door creaked open at 3 AM. I locked it.", "Fear"),
    ("Neighbor’s dog won’t stop barking at my closet.", "Fear"),
    ("Breathing in the backseat. Mirror showed nothing.", "Fear"),
    ("Static on the baby monitor. Then a laugh.", "Fear"),
    ("Handprint on the fogged shower glass. From inside.", "Fear"),
    ("TV turned on by itself. Volume maxed. Silence after.", "Fear"),
    ("Fingernail scratches on the attic door. No pets.", "Fear"),
    ("Got a text from my own number: ‘LOOK BEHIND YOU.’", "Fear"),
    ("The elevator stopped. Something knocked back.", "Fear"),
    ("Dreamt of drowning. Woke up soaked.", "Fear"),
    ("Camera caught a figure in my room. I was ‘alone.’", "Fear"),
    ("Neighbor vanished. Left their dinner half-eaten.", "Fear"),
    ("My reflection smiled. I didn’t.", "Fear"),
    ("Basement door won’t stay shut. Something’s pushing.", "Fear"),
    ("Sleep paralysis again. This time, it touched me.", "Fear"),

    # Aggressive (20 texts)
    ("I’ll break your nose!", "Aggressive"),
    ("Try me, you coward. I’ll end you.", "Aggressive"),
    ("Steal from me again and I’ll burn your life down.", "Aggressive"),
    ("Your face is my punching bag.", "Aggressive"),
    ("One more word and I’ll shut your mouth permanently.", "Aggressive"),
    ("I’ll carve my name into your bones.", "Aggressive"),
    ("You’re dead where you stand.", "Aggressive"),
    ("I’ll rip your tongue out for lying.", "Aggressive"),
    ("Hope you like hospital food, ’cause you’re going there.", "Aggressive"),
    ("I’ll make you beg for mercy.", "Aggressive"),
    ("Your skull’s about to meet concrete.", "Aggressive"),
    ("I’ll slice you open and laugh.", "Aggressive"),
    ("You’re a walking corpse.", "Aggressive"),
    ("I’ll make sure your family finds you in pieces.", "Aggressive"),
    ("I’ll break every bone you’ve got.", "Aggressive"),
    ("You’re not leaving this room whole.", "Aggressive"),
    ("I’ll put you six feet under.", "Aggressive"),
    ("Your blood’s gonna decorate my walls.", "Aggressive"),
    ("I’ll make you eat your teeth.", "Aggressive"),
    ("Pray I don’t find you tonight.", "Aggressive"),

    # Neutral (20 texts)
    ("Bought groceries. Ate cereal.", "Neutral"),
    ("Walked the dog. It pooped. Came home.", "Neutral"),
    ("Weather app says rain later. Maybe.", "Neutral"),
    ("Drank water. Checked mail. Nothing exciting.", "Neutral"),
    ("Traffic was normal. No honking.", "Neutral"),
    ("Washed dishes. Dried them. Put them away.", "Neutral"),
    ("Read two chapters. Forgot the plot.", "Neutral"),
    ("Took a nap. Woke up. Still tired.", "Neutral"),
    ("Scrolled social media. Closed it. Meh.", "Neutral"),
    ("Ate toast. It was okay.", "Neutral"),
    ("Clock ticks. Fridge hums. Silence.", "Neutral"),
    ("Typed an email. Deleted it.", "Neutral"),
    ("Clouds moved. Sun came out. Whatever.", "Neutral"),
    ("Checked the time. Checked it again.", "Neutral"),
    ("Tied my shoes. Untied them. Bored.", "Neutral"),
    ("Dusty shelves. Didn’t clean them.", "Neutral"),
    ("Folded laundry. Unfolded it. Sighed.", "Neutral"),
    ("Watered plants. They didn’t die.", "Neutral"),
    ("Stared at the wall. Thought nothing.", "Neutral"),
    ("Existed. That’s it.", "Neutral")
]

In [11]:
import pandas as pd
df = pd.DataFrame(texts, columns=["text", "sentiment"])
df

,text,sentiment
0,"Oh fuck, it’s coming!",Fear
1,"Hell yeah, I won!",Happiness
2,I miss you so much.,Sadness
3,You fucking suck!,Aggression
4,It’s just another day.,Neutrality
...,...,...
125,Saw a double rainbow after the storm. Magic.,Happy
126,Best friend’s baby giggled at me for the first...,Happy
127,Ate the spiciest ramen and survived. Victory d...,Happy
128,Grandma’s 90th birthday—three generations laug...,Happy


In [12]:
import time
import pandas as pd
results = []

for text in df['text']:
    start_time = time.time()
    cleaned_text = basic_text_cleaning(text)
    predicted_sentiment_index, labels = model.predict(cleaned_text)
    end_time = time.time()
    time_took = end_time - start_time
    length = len(text.split())
    
    results.append({
        "text": text,
        "predicted_sentiment": predicted_sentiment_index,
        "original sentiment": df[df['text'] == text]['sentiment'].values[0],
        "labels": labels,
        "time_took": time_took,
        "word count": length
    })
    
ans_df = pd.DataFrame(results)

In [13]:
ans_df

,text,predicted_sentiment,original sentiment,labels,time_took,word count
0,"Oh fuck, it’s coming!",Neutral,Fear,"{'Aggressive': 0.03, 'Fear': 0.52, 'Happy': 2....",0.449178,4
1,"Hell yeah, I won!",Happy,Happiness,"{'Aggressive': 0.65, 'Fear': 1.34, 'Happy': 75...",0.182868,4
2,I miss you so much.,Sad,Sadness,"{'Aggressive': 2.8, 'Fear': 18.51, 'Happy': 23...",0.184886,5
3,You fucking suck!,Happy,Aggression,"{'Aggressive': 1.32, 'Fear': 1.58, 'Happy': 78...",0.166622,3
4,It’s just another day.,Neutral,Neutrality,"{'Aggressive': 1.0, 'Fear': 0.91, 'Happy': 14....",0.232894,4
...,...,...,...,...,...,...
125,Saw a double rainbow after the storm. Magic.,Sad,Happy,"{'Aggressive': 1.1, 'Fear': 29.2, 'Happy': 3.4...",0.447650,8
126,Best friend’s baby giggled at me for the first...,Happy,Happy,"{'Aggressive': 0.34, 'Fear': 0.42, 'Happy': 82...",1.081271,12
127,Ate the spiciest ramen and survived. Victory d...,Neutral,Happy,"{'Aggressive': 0.18, 'Fear': 0.28, 'Happy': 2....",0.717314,9
128,Grandma’s 90th birthday—three generations laug...,Neutral,Happy,"{'Aggressive': 0.1, 'Fear': 0.03, 'Happy': 2.2...",0.432430,8


In [14]:
ans_df.to_csv('outputs.csv', index=False)

In [7]:
!pip freeze > requirements.txt 

In [45]:
text = """
The Forge of Fury

In the grimy shithole of Ironspike, where the air stank of sweat and molten steel, there was a forge that burned like a bastard’s hell. It was Korr’s domain, a hulking son of a bitch with fists that could smash skulls and a temper that’d fuck up anyone dumb enough to look at him wrong. He hammered out swords that’d gut you clean and axes that’d chop your damn head off, and he didn’t take shit from nobody—not the greedy-ass merchants, not the piss-drunk fools who staggered near his flames.

One scorching day, a cocky prick named Gav rolled into town—scarred-up, skinny as a rat, and grinning like he owned the fucking place. He stormed into Korr’s forge and slammed a dented breastplate down, barking, “Fix this, you slow-ass bastard, I’ve got a fight to win!” Korr’s eyes turned to slits, his hammer freezing mid-swing. “You don’t fucking tell me what to do, you little shit,” he snarled, stepping up so his bulk could crush Gav’s balls just by standing there.

The air was thick with hate. Gav didn’t back down—he spat a glob of phlegm on the floor and sneered, “I’ve fucked up bigger assholes than you.” That lit the fuse. Korr roared like a goddamn bull, chucking his hammer at Gav’s smug face. It missed by a cunt hair, smashing a rack of spears to splinters. Gav charged, pulling a nasty-ass dagger, and the forge turned into a fucking warzone. Sparks flew as they went at it—Korr swinging red-hot tongs like a madman, Gav slashing like a rabid dog. The townsfolk hauled ass out of there, their screams lost in the fire’s roar.

Blood hit the floor. Korr smashed Gav into the anvil, cracking his ribs like twigs, but Gav jammed his blade into Korr’s shoulder, blood hissing like piss on the coals. They tore into each other, no mercy—Korr’s fists pounding, Gav’s knife slicing. A table got fucked, the bellows exploded, and the flames shot up, ready to burn the whole damn place down. “I’ll rip your fucking head off!” Korr bellowed, grabbing Gav’s throat. Gav laughed like a psycho, smashing his forehead into Korr’s nose—crunch, blood everywhere.

It ended when Korr, half-blind with rage, hoisted Gav up and threw him into the forge. Gav’s scream was cut short as the fire ate him alive, his dagger clattering uselessly. Korr stood there, chest heaving, blood dripping, the forge a smoking ruin. “Fucking done,” he spat, kicking the breastplate into the ashes. Ironspike wouldn’t forget that day—Korr’s fury was the law now, and no bastard would dare test it again.
"""

print(len(text.split(" ")))

431
